In [ ]:
import os
import sys
cwd = os.getcwd()
src_path = os.path.join(cwd, "src")
sys.path.append(src_path)

In [6]:
from feature_extraction import extract_features_from_parse, dict2parse
from causal_model_features import build_cb_causal_model, build_sc_causal_model

# construct Causal Models
CM_MDL = build_cb_causal_model()
SC_MDL = build_sc_causal_model()

# 'Propn_fwd': 0.6,
# 'Propn_unique_codes': 0.7,
# 'Propn_unreachable': 0.2,

crel2probs = {
    "Causer:1->Result:50": [0.5, 0.6],
    "Causer:7->Result:50": [0.5, 0.6],
    "Causer:3->Result:4": [0.6],
    "Causer:4->Result:5": [0.6],
    "Causer:5->Result:4": [0.65],
    
    "Causer:11->Result:1": [0.35],
}

parse = dict2parse(crel2probs)
extract_features_from_parse(parse, crel2probs, CM_MDL)

defaultdict(float,
            {'Above-%-0.2': 1.0,
             'Above-%-0.3': 1.0,
             'Above-%-0.5': 0.8333333333333334,
             'Above-%-0.7': 0.0,
             'Above-%-0.8': 0.0,
             'Above-%-0.9': 0.0,
             'Above-%-0.95': 0.0,
             'Above-0.2': 6,
             'Above-0.3': 6,
             'Above-0.5': 5,
             'Above-0.7': 0,
             'Above-0.8': 0,
             'Above-0.9': 0,
             'Above-0.95': 0,
             'Above-All-Above-0.2': 1,
             'Above-All-Above-0.3': 1,
             'CChain-:11,1,50': 1,
             'CChain-:3,4,5': 1,
             'CChainStats-MaxChain_Len=3': 1,
             'CChainStats-crels_per_distinct_chain': 3.0,
             'CChainStats-num_distinct_chains=2': 1,
             'CREL_11:1': 1,
             'CREL_1:50': 1,
             'CREL_3:4': 1,
             'CREL_4:5': 1,
             'CREL_5:4': 1,
             'CREL_7:50': 1,
             'CREL_Causer:1->Result:50-MAX(prob)': 0.6,


In [ ]:
from collections import defaultdict
from pprint import pprint

def build_cb_causal_model():
    cm = dict()
    for i in range(1,5):
        cm[str(i)] = str(i+1)
    cm["5"] = "5b"
    cm["5b"] = "14"
    cm["14"] = "6"
    cm["6"] = "7"
    cm["7"] = "50"
    for i in range(11,14):
        cm[str(i)] = str(i+1)
    return cm

def build_sc_causal_model():
    cm = dict()
    for i in range(1,6):
        cm[str(i)] = str(i+1)
    cm["11"] = "12"
    cm["12"] = "6"
    cm["6"] = "50"
    return cm

build_sc_causal_model()

In [ ]:
# from typing import Dict

# def to_numeric_tag(tag: str)->int:
#     nums = ""
#     for c in tag:
#         if c.isdigit():
#             nums += c
#     return int(nums)

# TERMINAL = "50"
# cb_mdl = build_cb_causal_model()
# sc_mdl = build_sc_causal_model()

# def distance_between(a:str, b:str, causal_model: Dict[str, str]):
#     # ensure a <= b
#     a, b = str(a), str(b)
#     assert a in causal_model or a == TERMINAL, "a is not in the model"
#     assert b in causal_model or b == TERMINAL, "b is not in the model"    
    
#     # generally want this to be in order, but there are some exceptions
#     if (to_numeric_tag(a) > to_numeric_tag(b)):
#         a, b = b, a
    
#     diff = __nodes_between__(a, b, causal_model)
#     if diff == -1:
#         # try reverse order
#         return __nodes_between__(b, a, causal_model)
#     return diff

# def __nodes_between__(a, b, causal_model):
#     nodes_btwn = 0
#     current = a
#     while current != b and current != TERMINAL:
#         current = causal_model[current]
#         nodes_btwn += 1
#     # couldn't reach the other node
#     if b != TERMINAL and current == TERMINAL:
#         return -1
#     return nodes_btwn

# def is_forward_relation(causer, effect, causal_model):
#     causer, effect = str(causer), str(effect)
#     return __nodes_between__(causer, effect, causal_model) > 0

# # distance_between("11","6", sc_mdl)
# is_forward_relation("50","7", cb_mdl)